# Actual Data

In [1]:
from cov_est import *
from performance import *
import matplotlib.pyplot as plt

In [2]:
stock = pd.read_csv('stock.csv')
factors = pd.read_csv('factors.csv')

In [3]:
N = 1005   # period
delta = 1 / 252
train_ratio = 0.8
train_index = int(N*train_ratio)

In [4]:
ret_df = stock.pivot_table(values='ret', columns='cusip', index='date')
ret = ret_df[-N:].dropna(axis=1, how='any').values

In [5]:
r_train = ret[:train_index]
r_test = ret[train_index:]

In [6]:
f = factors.iloc[-N:,1:][:train_index].values

In [8]:
res = eval((r_train,r_test), delta, f=f, bench_f=f[:, 0])
display(res)

/Users/shuoyi/Optimization-Projects/performance.py:51: RuntimeWarning: invalid value encountered in log
  return (x.T@covmat@x/2-np.ones(N)@np.log(x)/N)**2


,minvar_in,minvar_out,maxdiv_in,maxdiv_out,riskpar_in,riskpar_out
sample,0.769042,-1.742785,1.405641,1.206740,1.139381,0.904059
const,0.281270,-1.540993,0.654171,0.524806,0.661269,0.553200
single_fac,0.616962,-0.737081,1.687746,1.233348,1.397599,0.972042
multi_fac,0.751591,0.924133,1.523235,1.132081,1.052627,1.047824


# Artificial Data

In [10]:
N = 252*2   # period
T = 2       # Year
k = 10      # num of factors
p = 30      # num of securities
S0 = np.random.randn(p) + 10

train_ratio = 0.8
train_index = int(N*train_ratio)


''' without factor, using geometric Brownian Motion here '''
rf = 0.03
np.random.seed(0)

BM_corr = pd.DataFrame(np.random.randn(20, p)).corr().to_numpy()
sig = np.diag(np.random.uniform(0.15, 0.3, p))
BM_cov = sig @ BM_corr @ sig
delta = T / N
S = np.zeros((N+1, p))
S[0] = S0
z = np.random.multivariate_normal(np.zeros(p), BM_cov, N)
for i in range(N):
    S[i+1] = S[i] * np.exp((rf-(BM_cov**2).sum(axis=1)/2)*delta + np.sqrt(delta)*z[i])
r = (S[1:] - S[:-1]) / S[:-1]


r_train = r[:train_index]
r_test = r[train_index:]
res_nonlinear = eval((r_train,r_test),delta)
print('Without linear factor structure')
display(res_nonlinear)

''' with linear factor structure '''
f = r[:,np.random.choice(p,k)]

beta = np.random.uniform(-1, 1, (k, p))
u = np.random.randn(N, p) / 1000
r = f @ beta + u

r_train = r[:train_index]
r_test = r[train_index:]
res_linear = eval((r_train,r_test),delta,f[:train_index])
print('-'*80)
print('With linear factor structure')
display(res_linear)

Without linear factor structure


,minvar_in,minvar_out,maxdiv_in,maxdiv_out,riskpar_in,riskpar_out
sample,1.644110,2.617307,1.615133,2.584773,0.957978,1.922039
const,1.188160,1.491080,0.898443,1.507911,0.898508,1.507981
single_fac,0.264404,1.182479,0.254209,1.327665,0.565905,1.364096
multi_fac,1.390444,1.587226,1.189265,1.551987,1.190071,1.534237


--------------------------------------------------------------------------------
With linear factor structure


,minvar_in,minvar_out,maxdiv_in,maxdiv_out,riskpar_in,riskpar_out
sample,0.324135,0.459263,0.216359,0.972021,0.388957,-0.040061
const,1.919394,1.088853,2.034555,1.496312,2.030639,1.495020
single_fac,0.724936,-2.370988,0.824505,-1.888804,0.964412,-1.439753
multi_fac,1.863066,1.065654,1.981132,1.496309,1.983173,1.499647
